In [1]:
import os
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# from scripts.preprocessing import preprocess_data


In [2]:
os.chdir("../")

In [3]:
train_data = pd.read_csv("data/rossmann/train.csv")
store_data = pd.read_csv("data/rossmann/store.csv")

C:\Users\windows 10\AppData\Local\Temp\ipykernel_13424\2941308998.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv("data/rossmann/train.csv")


In [4]:
df = pd.merge(train_data, store_data, on='Store')


In [5]:
df = df.head(100000)

In [6]:
from scripts.preprocessing import preprocess_data


df, label_encoders, scaler = preprocess_data(df)



c:\Users\windows 10\Desktop\Pharmaceuticals-_machine_learning\Pharmaceuticals-_machine_learning\scripts\preprocessing.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CompetitionDistance'].fillna(df['CompetitionDistance'].max() + 1, inplace=True)
c:\Users\windows 10\Desktop\Pharmaceuticals-_machine_learning\Pharmaceuticals-_machine_learning\scripts\preprocessing.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will chan

In [7]:
# Handle infinite and NaN values in X
X = df.drop(columns=['Sales'])  # Drop the 'Sales' column to use the remaining as features
X = X.replace([np.inf, -np.inf], np.nan)  # Replace infinity values with NaN
X = X.fillna(X.mode())  # Fill NaN values with the mode of the column

y = df['Sales']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns  # Identify string columns

# Define preprocessor with ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),  # Scale numerical columns
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # OneHotEncode categorical columns
])

# Build a pipeline with preprocessor and RandomForestRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocess both numerical and categorical features
    ('regressor', RandomForestRegressor(random_state=42))  # RandomForest model
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

# Output the results

print("Root Mean Squared Error (RMSE):", rmse)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')


Root Mean Squared Error (RMSE): 544.3467548457876
Mean Squared Error (MSE): 296313.38951113995
R² Score: 0.977364892043033


In [23]:
from sklearn.inspection import permutation_importance

# Calculate permutation importance
results = permutation_importance(pipeline, X_test, y_test, n_repeats=30, random_state=42)

# Create a DataFrame to store feature importances
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': results.importances_mean})

# Sort features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
import datetime
import pickle

model_filename = f"data/model_{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

new_data = pd.read_csv("data/test.csv   ")
predictions = loaded_model.predict(new_data)